In [18]:
import numpy as np
import matplotlib.pyplot as plt
import os
import imageio
import pandas as pd
from tqdm import tqdm
from skimage import io

from FF.image_processing import thresh, prepare_scalemap, edgedetector, measure_D
from FF.fractal_generation import midpoint_displacement, mountainpro

from NNIS import Network

import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [ ]:
def generate_coastline(D, iterations, P = 1):
        
        meshmap = midpoint_displacement(iterations, P = P, D=D)
        scalemap = prepare_scalemap(meshmap)
        threshmap, bwratio = thresh(scalemap, iterations)
        coastline = edgedetector(threshmap, iterations)
        return coastline

def generate_mountain(D, iterations, P = 1, zslice = 0.5):
        meshmap = midpoint_displacement(iterations, P = P, D=D)
        scalemap = prepare_scalemap(meshmap)
        slice, mountain = mountainpro(scalemap, iterations, zslice)
        return mountain

def generate_branch_network(network_params, neuron_params):
    
    network_id = network_params.get('network_id')
    network_width = network_params.get('width')
    network_height = network_params.get('height')
    num_neurons = network_params.get('num_neurons')

    # Create and generate the network
    network = Network(network_width, network_height, num_neurons, neuron_params, network_id)
    network.seed_neurons()
    network.grow_network()

    # Generate the binary mask
    network_mask = network.generate_binary_mask()

    return network_mask

In [7]:
def batch_generate_fractals(num_fractals, iterations, D_range, P,
                            fractal_type='coastline',
                            output_dir='fractals_batch',
                            neuron_params=None, network_params=None,
                            zslice=0.5):
    """
    Generates multiple fractal images with varying D values and saves them in the specified directory.

    :param num_fractals: Number of fractals to generate.
    :param iterations: Number of iterations for the fractal generation algorithms.
    :param D_range: Tuple (min_D, max_D) defining the range of D values.
    :param P: The parameter controlling the randomness in the midpoint displacement.
    :param fractal_type: The type of fractal to generate ('coastline', 'mountain', 'branch network').
    :param output_dir: Directory where the generated fractal images will be saved.
    :param neuron_params: Dictionary of parameters for neuron generation (used for 'branch network' type).
    :param network_params: Dictionary of parameters for network generation (used for 'branch network' type).
    :param zslice: The z-slice value for mountain generation (used for 'mountain' type).
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    min_D, max_D = D_range
    D_values = np.linspace(min_D, max_D, num_fractals)

    data = []

    for i, D in enumerate(tqdm(D_values, desc="Generating Fractals", unit="fractal")):

        if fractal_type == 'coastline':
            fractal = generate_coastline(D, iterations, P)

        elif fractal_type == 'mountain':
            fractal = generate_mountain(D, iterations, P, zslice)
            
        elif fractal_type == 'branch network':
            if neuron_params is None or network_params is None:
                return
            fractal = generate_branch_network(network_params, neuron_params)*255
            
        else:
            print('Invalid fractal type chosen, aborting.')
            return

        # Define the filename and save the fractal as a TIFF image
        #tiff_file = os.path.join(output_dir, f"{fractal_type}_fractal_{i}.tif")

        D_measured = measure_D(fractal, min_size = 8,max_size= np.shape(fractal)[0]//5, n_sizes=20, invert=False,)

        tiff_file = os.path.join(output_dir, f"{fractal_type}_fractal_{D_measured}.tif")
        imageio.imwrite(tiff_file, fractal.astype(np.uint8))  # Ensure the image is in uint8 format

        # Add the filename and D value to the list
        data.append([os.path.basename(tiff_file), D_measured, fractal_type])

    # Save the filenames and D values to a CSV file
    csv_file = os.path.join(output_dir, f"{fractal_type}_labels.csv")
    df = pd.DataFrame(data, columns=['filename', 'd_value', 'fractal_type'])
    df.to_csv(csv_file, index=False)

    print(f'Batch generation complete. {num_fractals} {fractal_type} fractals saved to {output_dir}.')
    print(f'Labels saved to {csv_file}.')

In [ ]:
neuron_params = {
    'depth': 3,
    'mean_soma_radius': 60,
    'std_soma_radius': 15,
    'D': 1.2,
    'branch_angle': np.pi / 4,
    'mean_branches': 1.5,
    'weave_type': 'Gauss',
    'randomness': 0.2,
    'curviness': 'Gauss',
    'curviness_magnitude': 1.5,
    'n_primary_dendrites': 5,
}

network_params = {
    'width': 2048,
    'height': 2048,
    'num_neurons': 10,
    'network_id': 'test'
}

batch_generate_fractals(
    num_fractals=500,  # Adjust the number as needed
    iterations=8,     # Not used in branch network but required by the function signature
    D_range=(1.1, 1.7),  # D is not used directly in branch network generation
    P=0.5,               # Not used in branch network but required by the function signature
    fractal_type='branch network',
    output_dir='/home/apd/Projects/FractalFluency/datasets/data_dump',
    neuron_params=neuron_params,
    network_params=network_params
)


In [16]:
def batch_generate_fractals_multi(num_fractals_per_type, 
                                  iterations, 
                                  D_ranges, 
                                  P,
                                  fractal_types=['coastline', 'mountain', 'branch network'],
                                  output_dir='/home/apd/Projects/FractalFluency/datasets/data_dump',
                                  neuron_params=None,
                                  network_params=None,
                                  zslice=0.5):
    
    """
    Generates multiple fractal images for multiple types with varying D values and saves them in the specified directory.

    :param num_fractals_per_type: Number of fractals to generate per type.
    :param iterations: Number of iterations for the fractal generation algorithms.
    :param D_ranges: Dictionary mapping fractal types to their (min_D, max_D) ranges.
                      Example: {'coastline': (1.1, 1.7), 'mountain': (1.2, 1.8), 'branch network': None}
    :param P: The parameter controlling the randomness in the midpoint displacement.
    :param fractal_types: List of fractal types to generate ('coastline', 'mountain', 'branch network').
    :param output_dir: Directory where the generated fractal images will be saved.
    :param neuron_params: Dictionary of parameters for neuron generation (used for 'branch network' type).
    :param network_params: Dictionary of parameters for network generation (used for 'branch network' type).
    :param zslice: The z-slice value for mountain generation (used for 'mountain' type).
    """


    os.makedirs(output_dir, exist_ok=True)
    data = []

    skipped = 0
    


    for fractal_type in fractal_types:
        print(f"\nGenerating fractals of type: {fractal_type}")

        # Retrieve D_range for the current fractal type
        D_range = D_ranges.get(fractal_type, None)

        if D_range:
            min_D, max_D = D_range
            D_values = np.linspace(min_D, max_D, num_fractals_per_type)
        else:
            # Assign a default D value (e.g., 0) if D is not applicable
            D_values = [0] * num_fractals_per_type

        for i, D in enumerate(tqdm(D_values, desc=f"Generating {fractal_type} fractals", unit="fractal")):

            if fractal_type == 'coastline':
                fractal = generate_coastline(D, iterations, P)

            elif fractal_type == 'mountain':
                fractal = generate_mountain(D, iterations, P, zslice)*255
            
            elif fractal_type == 'branch network':
                
                if neuron_params is None or network_params is None:
                    print('neuron_params and network_params must be provided for branch network generation. Skipping...')
                    skipped += 1
                    continue

                fractal = generate_branch_network(network_params, neuron_params)
                fractal = (fractal * 255).astype(np.uint8)  
            
            else:
                print(f'Invalid fractal type "{fractal_type}" chosen. Skipping...')
                skipped += 1
                continue 

            D_measured = measure_D(fractal, min_size=8, max_size=np.shape(fractal)[0]//5, n_sizes=20, invert=False)
            tiff_file = os.path.join(output_dir, f"{fractal_type}_{i}_{D_measured:.5f}.tif") 
            io.imsave(tiff_file, fractal.astype(np.uint8))  

            data.append([os.path.basename(tiff_file), D_measured, fractal_type])

    # Save all labels to a single CSV file
    csv_file = os.path.join(output_dir, 'labels.csv')
    df = pd.DataFrame(data, columns=['filename', 'd_value', 'fractal_type'])
    df.to_csv(csv_file, index=False)

    total_fractals = num_fractals_per_type * len(fractal_types) - skipped
    print(f'\nBatch generation complete, with {skipped} skipped. \n{total_fractals} fractals saved to {output_dir}.')
    print(f'Labels saved to {csv_file}.')

In [19]:
neuron_params = {
    'depth': 3,
    'mean_soma_radius': 60,
    'std_soma_radius': 15,
    'D': 1.2,
    'branch_angle': np.pi / 4,
    'mean_branches': 1.5,
    'weave_type': 'Gauss',
    'randomness': 0.2,
    'curviness': 'Gauss',
    'curviness_magnitude': 1.5,
    'n_primary_dendrites': 5,
}

network_params = {
    'width': 2048,
    'height': 2048,
    'num_neurons': 10,
    'network_id': 'test'
}

fractal_types = ['coastline', 'mountain', 'branch network']


D_ranges = {
    'coastline': (1.1, 1.9),
    'mountain': (1.1, 1.9),
    'branch network': None  # D is not applicable YET!!!
}


batch_generate_fractals_multi(
    num_fractals_per_type=500,  # Number of fractals per type
    iterations=9,
    D_ranges=D_ranges,
    P=1,
    fractal_types=fractal_types,
    output_dir='/home/apd/Projects/FractalFluency/datasets/first_batch',
    neuron_params=neuron_params,
    network_params=network_params,
    zslice=0.5
)


Generating fractals of type: coastline


Generating coastline fractals: 100%|██████████| 500/500 [06:09<00:00,  1.35fractal/s]



Generating fractals of type: mountain


Generating mountain fractals: 100%|██████████| 500/500 [14:58<00:00,  1.80s/fractal]   



Generating fractals of type: branch network


Generating branch network fractals: 100%|██████████| 500/500 [06:28<00:00,  1.29fractal/s]


Batch generation complete, with 0 skipped. 
1500 fractals saved to /home/apd/Projects/FractalFluency/datasets/first_batch.
Labels saved to /home/apd/Projects/FractalFluency/datasets/first_batch/labels.csv.
